# Task 1: Archiving the Company Listing
The purpose of this task is to scrape the Company Registration Office (CRO) website for company details and store them in a MySQL database. We will retrieve information for registration numbers ranging from 1 to 337078.

Source: 'https://application.ocr.gov.np/faces/CompanyDetails.jsp'

In [1]:
import asyncio
import aiohttp
import logging
from bs4 import BeautifulSoup
import aiomysql
from datetime import datetime

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
# Function to validate date
def validate_date(date_text):
    if date_text:
        try:
            return datetime.strptime(date_text, '%Y-%m-%d').date()
        except ValueError:
            return None
    return None

In [3]:
async def create_pool():
    try:
        pool = await aiomysql.create_pool(
            host="localhost",
            port=3306,
            user="root",
            password="Mysql@2024",
            db="employers_rating",
            autocommit=True,
            auth_plugin='mysql_native_password',
            minsize=1,
            maxsize=10
        )
        logging.info("Database connection pool created successfully")
        return pool
    except aiomysql.Error as err:
        logging.error(f"Database connection error: {err}")
        return None

In [4]:
async def create_table_if_not_exists(pool):
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            await cursor.execute("""
            CREATE TABLE IF NOT EXISTS company_master (
                COMPANY_ID INT AUTO_INCREMENT PRIMARY KEY,
                CRO_SN INT,
                CRO_Nepali_Name VARCHAR(255),
                CRO_English_Name VARCHAR(2555),
                CRO_Registration_Number INT UNIQUE,
                CRO_Registration_Date_BS DATE,
                CRO_Company_Type VARCHAR(255),
                CRO_Company_Address VARCHAR(255),
                CRO_Recent_Communication_With_OCR DATE
            )
            """)

In [5]:
async def insert_or_update_data(pool, data):
    async with pool.acquire() as conn:
        async with conn.cursor() as cursor:
            await cursor.execute("""
            INSERT INTO company_master (CRO_SN, CRO_Nepali_Name, CRO_English_Name, CRO_Registration_Number, CRO_Registration_Date_BS, CRO_Company_Type, CRO_Company_Address, CRO_Recent_Communication_With_OCR)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ON DUPLICATE KEY UPDATE
            CRO_SN = VALUES(CRO_SN),
            CRO_Nepali_Name = VALUES(CRO_Nepali_Name),
            CRO_English_Name = VALUES(CRO_English_Name),
            CRO_Registration_Date_BS = VALUES(CRO_Registration_Date_BS),
            CRO_Company_Type = VALUES(CRO_Company_Type),
            CRO_Company_Address = VALUES(CRO_Company_Address),
            CRO_Recent_Communication_With_OCR = VALUES(CRO_Recent_Communication_With_OCR)
            """, data)

In [6]:
HEADERS = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-US,en;q=0.9',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
}

url = 'https://application.ocr.gov.np/faces/CompanyDetails.jsp'

In [7]:


async def submit_form_and_process(session, registration_number, pool):
    try:
        async with session.get(url, headers=HEADERS, ssl=False) as response:
            response.raise_for_status()
            soup = BeautifulSoup(await response.text(), 'html.parser')
            form = soup.find('form')

            if not form:
                logging.warning("Form not found in the page.")
                return

            action = form.get('action')
            if not action:
                logging.warning("Form action attribute not found.")
                return

            action_url = url if action.startswith('/') else url + action

            payload = {input_field.get('name'): input_field.get('value', '') for input_field in form.find_all('input')}
            payload['j_id_jsp_826405674_6:registrationNumber'] = str(registration_number)

            logging.info(f"Submitting form to {action_url} with payload: {payload}")

            async with session.post(action_url, data=payload, headers=HEADERS, ssl=False) as post_response:
                post_response.raise_for_status()
                logging.info(f"Form submitted successfully for registration number {registration_number}.")

                soup = BeautifulSoup(await post_response.text(), 'html.parser')
                table = soup.find('table', {'id': 'j_id_jsp_826405674_6:companyDetails'})

                if table:
                    rows = table.find_all('tr')
                    for row in rows:
                        cols = row.find_all('td')
                        if len(cols) == 8:
                            data = [col.text.strip() if col.text.strip() != '' else None for col in cols]
                            data[4] = validate_date(data[4]) if data[4] else None
                            data[7] = validate_date(data[7]) if data[7] else None

                            await insert_or_update_data(pool, data)
                            logging.info("Data inserted or updated successfully.")
                else:
                    logging.warning(f"Table not found in the response for registration number {registration_number}.")
    except aiohttp.ClientError as e:
        logging.error(f"HTTP request error for registration number {registration_number}: {e}")
    except aiohttp.ClientConnectionError as e:
        logging.error(f"Connection error for registration number {registration_number}: {e}")
    except aiomysql.Error as e:
        logging.error(f"Database error for registration number {registration_number}: {e}")
    except Exception as e:
        logging.error(f"Unexpected error for registration number {registration_number}: {e}")

In [ ]:
async def main():
    pool = await create_pool()
    if pool:
        await create_table_if_not_exists(pool)

        async with aiohttp.ClientSession(cookie_jar=aiohttp.CookieJar()) as session:
            tasks = []
            for reg_num in range(1, 337078):  
                task = submit_form_and_process(session, reg_num, pool)
                tasks.append(task)
                if len(tasks) % 10 == 0:  # Adjust this number as needed
                    await asyncio.gather(*tasks)
                    tasks = []

            if tasks:
                await asyncio.gather(*tasks)

        pool.close()
        await pool.wait_closed()
        logging.info("Database connection pool closed.")

# Run the main function in Jupyter notebook
await main()

2024-07-10 13:31:37,431 - INFO - Database connection pool created successfully
C:\Users\KIIT01\anaconda3\Lib\site-packages\aiomysql\cursors.py:239: Warning: Table 'company_master' already exists
  await self._query(query)
2024-07-10 13:31:37,772 - INFO - Submitting form to https://application.ocr.gov.np/faces/CompanyDetails.jsp with payload: {'j_id_jsp_826405674_6': 'j_id_jsp_826405674_6', 'j_id_jsp_826405674_6:registrationNumber': '2', 'j_id_jsp_826405674_6:j_id_jsp_826405674_16': 'Search', 'javax.faces.ViewState': 'j_id27621:j_id27622'}
2024-07-10 13:31:37,777 - INFO - Submitting form to https://application.ocr.gov.np/faces/CompanyDetails.jsp with payload: {'j_id_jsp_826405674_6': 'j_id_jsp_826405674_6', 'j_id_jsp_826405674_6:registrationNumber': '1', 'j_id_jsp_826405674_6:j_id_jsp_826405674_16': 'Search', 'javax.faces.ViewState': 'j_id27623:j_id27624'}
2024-07-10 13:31:37,782 - INFO - Submitting form to https://application.ocr.gov.np/faces/CompanyDetails.jsp with payload: {'j_id_jsp